In [98]:
## Preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import pickle

## RNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


df = pd.read_csv('Churn_Modelling.csv', sep=',')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [99]:
## Data Preprocessing
data = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [100]:
## Encoding Categorical Data
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [101]:
# Create an instance of OneHotEncoder
one_hot_encoder_geo = OneHotEncoder()

# Reshape the 'Geography' column to be 2-dimensional
geo_reshaped = data['Geography'].values.reshape(-1, 1)

# Fit and transform the 'Geography' column
geography_encoded = one_hot_encoder_geo.fit_transform(geo_reshaped)


# Convert the result to a DataFrame (optional, based on your needs)
geography_encoded_df = pd.DataFrame(geography_encoded.toarray(), columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

# If you want to join it back to your original DataFrame
data = data.join(geography_encoded_df)
data = data.drop('Geography', axis=1)

In [102]:
# Save the encoder to a pickle file
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

# Save the encoder to a pickle file
with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)

In [103]:
## devide the dataset into depandent and independent variables

X = data.drop('Exited', axis=1)
y = data['Exited']

## Splitting the dataset into the Training set and Test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [104]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [105]:
## Building the RNN

model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ]
)

/Users/tafehi/personal/NLP/course/tutorial/nltk/ann/ann-project/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [106]:
## Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [107]:
# Set up the TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [108]:
## Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [109]:
## Train the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=100, 
                    callbacks=[tensorflow_callback, early_stopping_callback]) 

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7585 - loss: 0.5209 - val_accuracy: 0.8295 - val_loss: 0.4000
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.8242 - loss: 0.4216 - val_accuracy: 0.8475 - val_loss: 0.3706
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step - accuracy: 0.8306 - loss: 0.3993 - val_accuracy: 0.8575 - val_loss: 0.3504
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step - accuracy: 0.8437 - loss: 0.3705 - val_accuracy: 0.8615 - val_loss: 0.3489
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 670us/step - accuracy: 0.8524 - loss: 0.3643 - val_accuracy: 0.8600 - val_loss: 0.3427
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 0.8473 - loss: 0.3668 - val_accuracy: 0.8630 - val_loss: 0.3409
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - accuracy: 0.8543 - loss: 0.3581 - val_accuracy: 0.8625 - val_loss: 0.3457
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.8480 - loss: 0.3

In [112]:
model.save('model.h5')

In [110]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [111]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 14689), started 0:03:46 ago. (Use '!kill 14689' to kill it.)